<a href="https://colab.research.google.com/github/BimantoroSetyo/UAS-MI/blob/main/UAS_MI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install streamlit pandas plotly pyngrok

In [14]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px

# --- KONFIGURASI HALAMAN ---
st.set_page_config(
    page_title="Media Intelligence Dashboard",
    page_icon="📊",
    layout="wide"
)

# --- FUNGSI UNTUK MEMUAT DAN MEMBERSIHKAN DATA ---
@st.cache_data
def load_data(file_path='SwayTea.csv'):
    """
    Memuat data dari file CSV.
    Melakukan pembersihan dan transformasi data dasar.
    """
    df = pd.read_csv(file_path)

    # --- Pembersihan & Transformasi Data ---
    df['Date'] = pd.to_datetime(df['Date'])
    try:
        split_cols = df['Influencer_Brand'].str.split('|', expand=True)
        df['Influencer'] = split_cols[0].str.strip()
        df['Brand'] = split_cols[1].str.strip()
    except Exception as e:
        df['Influencer'] = 'N/A'
        df['Brand'] = 'N/A'
        st.warning(f"Could not split 'Influencer_Brand' column. Error: {e}")
    df.drop(columns=['Influencer_Brand'], inplace=True)

    return df

# --- UI: SIDEBAR UNTUK FILTER ---
# Di versi ini, kita tidak menggunakan fitur upload karena data sudah ada di repo.
# Jika Anda ingin tetap ada, Anda bisa menambahkan kembali logika upload file.
st.sidebar.header("📊 Filter Dashboard")

# Memuat data
df = load_data()

# Filter berdasarkan rentang tanggal
min_date = df['Date'].min().date()
max_date = df['Date'].max().date()
start_date, end_date = st.sidebar.date_input(
    "Pilih rentang tanggal:",
    value=(min_date, max_date),
    min_value=min_date,
    max_value=max_date
)
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Filter lainnya
selected_platforms = st.sidebar.multiselect("Pilih Platform:", options=df['Platform'].unique(), default=df['Platform'].unique())
selected_sentiments = st.sidebar.multiselect("Pilih Sentimen:", options=df['Sentiment'].unique(), default=df['Sentiment'].unique())
selected_locations = st.sidebar.multiselect("Pilih Lokasi:", options=df['Location'].unique(), default=df['Location'].unique())

# --- PENERAPAN FILTER KE DATAFRAME ---
df_filtered = df[
    (df['Date'] >= start_date) & (df['Date'] <= end_date) &
    (df['Platform'].isin(selected_platforms)) &
    (df['Sentiment'].isin(selected_sentiments)) &
    (df['Location'].isin(selected_locations))
]

# --- UI: DASHBOARD UTAMA ---
st.title("📊 Interactive Media Intelligence Dashboard")
st.markdown("Analisis Performa Kampanye SwayTea di Berbagai Media Sosial")

if df_filtered.empty:
    st.warning("Tidak ada data yang cocok dengan filter yang Anda pilih.")
else:
    # --- Ringkasan Metrik Utama (KPIs) ---
    total_posts = len(df_filtered)
    total_engagements = df_filtered['Engagements'].sum()
    avg_engagements = df_filtered['Engagements'].mean()

    col1, col2, col3 = st.columns(3)
    col1.metric("Total Posts", f"{total_posts:,}")
    col2.metric("Total Engagements", f"{total_engagements:,.0f}")
    col3.metric("Rata-rata Engagement/Post", f"{avg_engagements:,.2f}")

    st.markdown("---")

    # --- Visualisasi ---
    tab1, tab2, tab3 = st.tabs(["📈 Analisis Umum", "🧑‍💻 Analisis Detail", "📄 Data Mentah"])

    with tab1:
        st.subheader("Tren Engagement dari Waktu ke Waktu")
        engagement_over_time = df_filtered.groupby('Date')['Engagements'].sum().reset_index()
        fig_time = px.line(engagement_over_time, x='Date', y='Engagements', title='Total Engagement Harian')
        st.plotly_chart(fig_time, use_container_width=True)

        col1, col2 = st.columns(2)
        with col1:
            st.subheader("Distribusi Sentimen")
            sentiment_dist = df_filtered['Sentiment'].value_counts().reset_index()
            fig_pie = px.pie(sentiment_dist, names='Sentiment', values='count', title='Proporsi Sentimen Post', hole=0.3, color_discrete_map={'Positive':'#2ca02c', 'Neutral':'#ff7f0e', 'Negative':'#d62728'})
            st.plotly_chart(fig_pie, use_container_width=True)
        with col2:
            st.subheader("Engagement per Platform")
            platform_engagement = df_filtered.groupby('Platform')['Engagements'].sum().sort_values(ascending=False).reset_index()
            fig_bar_platform = px.bar(platform_engagement, x='Platform', y='Engagements', title='Total Engagement Berdasarkan Platform', text_auto=True)
            st.plotly_chart(fig_bar_platform, use_container_width=True)

    with tab2:
        col1, col2 = st.columns(2)
        with col1:
            st.subheader("Top Influencer berdasarkan Engagement")
            influencer_engagement = df_filtered.groupby('Influencer')['Engagements'].sum().sort_values(ascending=True).reset_index()
            fig_influencer = px.bar(influencer_engagement.tail(10), y='Influencer', x='Engagements', orientation='h', title='Top 10 Influencer', text_auto='.2s')
            st.plotly_chart(fig_influencer, use_container_width=True)
        with col2:
            st.subheader("Engagement per Tipe Post")
            post_type_engagement = df_filtered.groupby('Post_Type')['Engagements'].sum().sort_values(ascending=False).reset_index()
            fig_post_type = px.bar(post_type_engagement, x='Post_Type', y='Engagements', title='Total Engagement Berdasarkan Tipe Post', text_auto='.2s')
            st.plotly_chart(fig_post_type, use_container_width=True)

    with tab3:
        st.subheader("Data Mentah Sesuai Filter")
        st.dataframe(df_filtered)

Overwriting app.py
